In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import gc

In [3]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [55]:
feature = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data10year/feature.csv')
payoff = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data10year/payoff.csv')
race_info = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data10year/race_info.csv')
race_result = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data10year/race_result.csv')


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
feature_plus = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba-scraper-master-plus/plus_data/feature.csv')
payoff_plus = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba-scraper-master-plus/plus_data/payoff.csv')
race_info_plus = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba-scraper-master-plus/plus_data/race_info.csv')
race_result_plus = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba-scraper-master-plus/plus_data/race_result.csv')

In [58]:
def concat_data(feature,payoff,race_info,race_result,feature_plus,payoff_plus,race_info_plus,race_result_plus):
    #最大のrace_idを足し算して被りをなくす
    n = race_info['id'].max()
    feature_plus['race_id'] += n
    payoff_plus['race_id'] += n
    race_info_plus['id'] += n
    race_result_plus['race_id'] += n
    #前のデータの最後の日付を取得　→　結合するデータをその日より後のデータだけにする
    last_date_of_pre_data = race_info.date.max()
    del_id_list = list(race_info_plus[race_info_plus['date']<=last_date_of_pre_data]['id'].unique()) #削除するデータのidリスト
    for del_id in del_id_list:
        feature_plus.drop(feature_plus.index[feature_plus.race_id == del_id], inplace=True)
        payoff_plus.drop(payoff_plus.index[payoff_plus.race_id == del_id], inplace=True)
        race_info_plus.drop(race_info_plus.index[race_info_plus.id == del_id], inplace=True)
        race_result_plus.drop(race_result_plus.index[race_result_plus.race_id == del_id], inplace=True)
    #結合
    new_feature = pd.concat([feature, feature_plus])
    new_payoff = pd.concat([payoff, payoff_plus])
    new_race_info = pd.concat([race_info, race_info_plus])
    new_race_result = pd.concat([race_result, race_result_plus])
    #index振り直し
    reidx_new_race_info = new_race_info.reset_index().drop('index',axis=1)
    #振り直したindex+1を、new_idとしてカラムに追加
    reidx_new_race_info = reidx_new_race_info.reset_index()
    reidx_new_race_info = reidx_new_race_info.rename(columns={'index': 'new_id' , 'id':'race_id'})
    reidx_new_race_info['new_id'] = reidx_new_race_info['new_id']+1
    #idが１〜綺麗に揃うように変更していく
    reidx_new_feature = pd.merge( reidx_new_race_info[['new_id', 'race_id']],new_feature,  on='race_id')
    reidx_new_feature = reidx_new_feature.drop('race_id',axis=1).rename(columns={'new_id':'race_id'})
    reidx_new_payoff = pd.merge( reidx_new_race_info[['new_id', 'race_id']],new_payoff,  on='race_id')
    reidx_new_payoff = reidx_new_payoff.drop('race_id',axis=1).rename(columns={'new_id':'race_id'})
    reidx_new_race_result = pd.merge( reidx_new_race_info[['new_id', 'race_id']],new_race_result,  on='race_id')
    reidx_new_race_result = reidx_new_race_result.drop('race_id',axis=1).rename(columns={'new_id':'race_id'})
    #race_infoのrace_idをidに直しておく
    reidx_new_race_info = reidx_new_race_info.drop('race_id',axis=1).rename(columns={'new_id':'id'})
    
    return reidx_new_feature, reidx_new_payoff, reidx_new_race_info, reidx_new_race_result

In [59]:
a,b,c,d = concat_data(feature,payoff,race_info,race_result,feature_plus,payoff_plus,race_info_plus,race_result_plus)

In [64]:
a.to_csv('feature.csv',index=False)
b.to_csv('payoff.csv',index=False)
c.to_csv('race_info.csv',index=False)
d.to_csv('race_result.csv',index=False)

In [67]:
pd.read_csv('feature.csv').tail()

,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,avgsr4,avgWin4,dhweight,disavesr,disRoc,distance,dsl,enterTimes,eps,hweight,jwinper,odds,owinper,preSRa,sex,surface,surfaceScore,twinper,weather,weight,winRun,jEps,jAvgWin4,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLateStart,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly
504745,34990,18,2,14.0,2013103969,1114,356,5.0,NaN,NaN,6.0,NaN,NaN,1200.0,NaN,0.0,NaN,470.0,0.00000,348.2,NaN,NaN,セ,芝,NaN,0.071429,晴,53.0,NaN,25.000000,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504746,34990,11,2,15.0,2014103884,894,427,4.0,NaN,NaN,3.0,NaN,NaN,1200.0,NaN,0.0,NaN,538.0,0.00000,256.1,0.0,NaN,牡,芝,NaN,0.000000,晴,53.0,NaN,18.928571,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504747,34990,12,2,16.0,2015103673,1156,1028,3.0,NaN,NaN,8.0,NaN,NaN,1200.0,NaN,0.0,NaN,430.0,0.00000,22.1,0.0,NaN,牝,芝,NaN,0.066667,晴,51.0,NaN,35.111111,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504748,34990,4,2,17.0,2013103801,1037,1078,5.0,NaN,NaN,12.0,NaN,NaN,1200.0,NaN,0.0,NaN,488.0,0.00000,50.4,0.0,NaN,牡,芝,NaN,0.181818,晴,56.0,NaN,68.304167,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504749,34990,7,2,18.0,2011105974,1018,1087,7.0,NaN,NaN,4.0,NaN,NaN,1200.0,NaN,0.0,NaN,528.0,0.06383,16.8,NaN,NaN,牝,芝,NaN,0.083333,晴,53.0,NaN,112.553191,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0


In [11]:
n = race_info['id'].max()
n

34439

In [12]:
feature_plus['race_id'] += n
payoff_plus['race_id'] += n
race_info_plus['id'] += n
race_result_plus['race_id'] += n

In [13]:
last_date_of_pre_data = race_info.date.max()
last_date_of_pre_data

'2018-10-14'

In [15]:
del_id_list = list(race_info_plus[race_info_plus['date']<=last_date_of_pre_data]['id'].unique())
del_id_list[0:6]

[34440, 34445, 34449, 34451, 34453, 34457]

In [16]:
for del_id in del_id_list:
    feature_plus.drop(feature_plus.index[feature_plus.race_id == del_id], inplace=True)
    payoff_plus.drop(payoff_plus.index[payoff_plus.race_id == del_id], inplace=True)
    race_info_plus.drop(race_info_plus.index[race_info_plus.id == del_id], inplace=True)
    race_result_plus.drop(race_result_plus.index[race_result_plus.race_id == del_id], inplace=True)

In [32]:
new_feature = pd.concat([feature, feature_plus])
new_payoff = pd.concat([payoff, payoff_plus])
new_race_info = pd.concat([race_info, race_info_plus])
new_race_result = pd.concat([race_result, race_result_plus])

In [34]:
#index振り直し
new_race_info = new_race_info.reset_index().drop('index',axis=1)
new_race_info.tail()

,id,race_name,surface,distance,weather,surface_state,race_start,race_number,surface_score,date,place_detail,race_class
34985,35142,2歳新馬,芝左,1600,晴,芝 : 良,12:45,6,NaN,2018-11-10,5回東京3日目,サラ系2歳新馬 [指](馬齢)
34986,35143,浦佐特別,芝左外,2000,晴,芝 : 良,14:40,10,NaN,2018-10-21,3回新潟4日目,サラ系3歳以上500万下 (混)[指](定量)
34987,35144,3歳上500万下,芝右,1200,曇,芝 : 良,14:10,9,NaN,2018-11-17,3回福島5日目,サラ系3歳以上500万下 (定量)
34988,35145,秋明菊賞,芝右外,1400,晴,芝 : 良,14:25,9,NaN,2018-11-18,5回京都6日目,サラ系2歳500万下 (混)(特指)(馬齢)
34989,35146,壬生特別,芝右,1200,晴,芝 : 良,14:25,9,NaN,2018-10-20,4回京都6日目,サラ系3歳以上1000万下 (混)[指](ハンデ)


In [35]:
#振り直したindexを、new_idとしてカラムに追加
new_race_info = new_race_info.reset_index()
new_race_info = new_race_info.rename(columns={'index': 'new_id' , 'id':'race_id'})
new_race_info['new_id'] = new_race_info['new_id']+1
new_race_info.tail()

,new_id,race_id,race_name,surface,distance,weather,surface_state,race_start,race_number,surface_score,date,place_detail,race_class
34985,34986,35142,2歳新馬,芝左,1600,晴,芝 : 良,12:45,6,NaN,2018-11-10,5回東京3日目,サラ系2歳新馬 [指](馬齢)
34986,34987,35143,浦佐特別,芝左外,2000,晴,芝 : 良,14:40,10,NaN,2018-10-21,3回新潟4日目,サラ系3歳以上500万下 (混)[指](定量)
34987,34988,35144,3歳上500万下,芝右,1200,曇,芝 : 良,14:10,9,NaN,2018-11-17,3回福島5日目,サラ系3歳以上500万下 (定量)
34988,34989,35145,秋明菊賞,芝右外,1400,晴,芝 : 良,14:25,9,NaN,2018-11-18,5回京都6日目,サラ系2歳500万下 (混)(特指)(馬齢)
34989,34990,35146,壬生特別,芝右,1200,晴,芝 : 良,14:25,9,NaN,2018-10-20,4回京都6日目,サラ系3歳以上1000万下 (混)[指](ハンデ)


In [40]:
_new_feature = pd.merge( new_race_info[['new_id', 'race_id']],new_feature,  on='race_id')
_new_feature = _new_feature.drop('race_id',axis=1).rename(columns={'new_id':'race_id'})
_new_feature.tail()

,race_id,horse_number,grade,order_of_finish,horse_id,jockey_id,trainer_id,age,avgsr4,avgWin4,dhweight,disavesr,disRoc,distance,dsl,enterTimes,eps,hweight,jwinper,odds,owinper,preSRa,sex,surface,surfaceScore,twinper,weather,weight,winRun,jEps,jAvgWin4,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLateStart,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly
504745,34990,18,2,14.0,2013103969,1114,356,5.0,NaN,NaN,6.0,NaN,NaN,1200.0,NaN,0.0,NaN,470.0,0.00000,348.2,NaN,NaN,セ,芝,NaN,0.071429,晴,53.0,NaN,25.000000,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504746,34990,11,2,15.0,2014103884,894,427,4.0,NaN,NaN,3.0,NaN,NaN,1200.0,NaN,0.0,NaN,538.0,0.00000,256.1,0.0,NaN,牡,芝,NaN,0.000000,晴,53.0,NaN,18.928571,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504747,34990,12,2,16.0,2015103673,1156,1028,3.0,NaN,NaN,8.0,NaN,NaN,1200.0,NaN,0.0,NaN,430.0,0.00000,22.1,0.0,NaN,牝,芝,NaN,0.066667,晴,51.0,NaN,35.111111,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504748,34990,4,2,17.0,2013103801,1037,1078,5.0,NaN,NaN,12.0,NaN,NaN,1200.0,NaN,0.0,NaN,488.0,0.00000,50.4,0.0,NaN,牡,芝,NaN,0.181818,晴,56.0,NaN,68.304167,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0
504749,34990,7,2,18.0,2011105974,1018,1087,7.0,NaN,NaN,4.0,NaN,NaN,1200.0,NaN,0.0,NaN,528.0,0.06383,16.8,NaN,NaN,牝,芝,NaN,0.083333,晴,53.0,NaN,112.553191,NaN,NaN,NaN,10.0,NaN,NaN,0.0,NaN,NaN,右,京都,18.0,NaN,NaN,NaN,NaN,0.0


In [ ]:
_new_feature = pd.merge( new_race_info[['new_id', 'race_id']],new_feature,  on='race_id')
_new_feature = _new_feature.drop('race_id',axis=1).rename(columns={'new_id':'race_id'})
_new_feature.tail()

In [131]:
new_feature.to_csv('feature.csv',index=False)
new_payoff.to_csv('payoff.csv',index=False)
new_race_info.to_csv('race_info.csv',index=False)
new_race_result.to_csv('race_result.csv',index=False)

In [55]:
def dammy(data , columns):
    for col in columns:
        dammy_df=pd.get_dummies(data[[col]].astype(object))
        data=pd.concat([data, dammy_df], axis=1)
        data=data.drop(col,axis=1)
        del dammy_df
        gc.collect()
    return data

In [57]:
columns = ['grade' , 'sex' , 'surface' , 'weather']

In [61]:
dammied = dammy(feature , columns)

In [63]:
dammied.head()

,race_id,horse_number,order_of_finish,horse_id,jockey_id,trainer_id,age,avgsr4,avgWin4,dhweight,disavesr,disRoc,distance,dsl,enterTimes,eps,hweight,jwinper,odds,owinper,preSRa,surfaceScore,twinper,weight,winRun,jEps,jAvgWin4,preOOF,pre2OOF,month,ridingStrongJockey,runningStyle,preLateStart,preLastPhase,lateStartPer,course,placeCode,headCount,preHeadCount,surfaceChanged,gradeChanged,preMargin,femaleOnly,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,sex_セ,sex_牝,sex_牡,surface_ダ,surface_芝,weather_他,weather_小雨,weather_晴,weather_曇,weather_雨,weather_雪
0,1,9,1.0,2006105662,1102,353,2.0,NaN,NaN,4.0,NaN,NaN,1200.0,NaN,0.0,NaN,452.0,NaN,7.0,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
1,1,11,2.0,2006105747,635,429,2.0,NaN,NaN,0.0,NaN,NaN,1200.0,NaN,0.0,NaN,440.0,NaN,4.1,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
2,1,1,3.0,2006102910,1122,1076,2.0,NaN,NaN,-6.0,NaN,NaN,1200.0,NaN,0.0,NaN,428.0,NaN,2.4,NaN,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
3,1,7,4.0,2006100792,679,106,2.0,NaN,NaN,-10.0,NaN,NaN,1200.0,NaN,0.0,NaN,460.0,NaN,9.7,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
4,1,14,5.0,2006102658,1117,1067,2.0,NaN,NaN,0.0,NaN,NaN,1200.0,NaN,0.0,NaN,420.0,NaN,5.6,NaN,NaN,NaN,NaN,51.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,0.0,NaN,NaN,右,福島,16.0,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0


In [75]:
feature.at[497179,'age'] = 2

In [69]:
race_id = 34438
eachRace = feature[feature['race_id']==race_id].reset_index()
for n in range(len(eachRace)) :
    idx = eachRace.loc[n,'index']
    print(idx)

497179
497180
497181
497182
497183
497184
497185
497186
497187
497188
497189
497190
497191
497192


In [50]:
for col in list(race_result.columns):
    print(col ,'::::',  type(race_result[col].values[0]))

race_id :::: <class 'numpy.int64'>
order_of_finish :::: <class 'str'>
frame_number :::: <class 'numpy.int64'>
horse_number :::: <class 'numpy.int64'>
horse_id :::: <class 'numpy.int64'>
sex :::: <class 'str'>
age :::: <class 'numpy.int64'>
basis_weight :::: <class 'numpy.float64'>
jockey_id :::: <class 'numpy.int64'>
finishing_time :::: <class 'str'>
length :::: <class 'float'>
speed_figure :::: <class 'numpy.float64'>
pass :::: <class 'str'>
last_phase :::: <class 'numpy.float64'>
odds :::: <class 'numpy.float64'>
popularity :::: <class 'numpy.float64'>
horse_weight :::: <class 'str'>
remark :::: <class 'float'>
stable :::: <class 'str'>
trainer_id :::: <class 'numpy.int64'>
owner_id :::: <class 'int'>
earning_money :::: <class 'numpy.float64'>


In [49]:
for col in list(race_info.columns):
    print(col ,'::::',  type(race_info[col].values[0]))

id :::: <class 'numpy.int64'>
race_name :::: <class 'str'>
surface :::: <class 'str'>
distance :::: <class 'numpy.int64'>
weather :::: <class 'str'>
surface_state :::: <class 'str'>
race_start :::: <class 'str'>
race_number :::: <class 'numpy.int64'>
surface_score :::: <class 'numpy.float64'>
date :::: <class 'str'>
place_detail :::: <class 'str'>
race_class :::: <class 'str'>


In [48]:
for col in list(feature.columns):
    print(col ,'::::',  type(feature[col].values[0]))

race_id :::: <class 'numpy.int64'>
horse_number :::: <class 'numpy.int64'>
grade :::: <class 'numpy.int64'>
order_of_finish :::: <class 'numpy.float64'>
horse_id :::: <class 'numpy.int64'>
jockey_id :::: <class 'numpy.int64'>
trainer_id :::: <class 'numpy.int64'>
age :::: <class 'numpy.float64'>
avgsr4 :::: <class 'numpy.float64'>
avgWin4 :::: <class 'numpy.float64'>
dhweight :::: <class 'numpy.float64'>
disavesr :::: <class 'numpy.float64'>
disRoc :::: <class 'numpy.float64'>
distance :::: <class 'numpy.float64'>
dsl :::: <class 'numpy.float64'>
enterTimes :::: <class 'numpy.float64'>
eps :::: <class 'numpy.float64'>
hweight :::: <class 'numpy.float64'>
jwinper :::: <class 'numpy.float64'>
odds :::: <class 'numpy.float64'>
owinper :::: <class 'numpy.float64'>
preSRa :::: <class 'numpy.float64'>
sex :::: <class 'str'>
surface :::: <class 'str'>
surfaceScore :::: <class 'numpy.float64'>
twinper :::: <class 'numpy.float64'>
weather :::: <class 'str'>
weight :::: <class 'numpy.float64'>
w

In [4]:
input_data = pd.read_csv('/Users/wasedafuuta/Desktop/keiba/netkeiba_data/data10year/inputdata.csv')

In [42]:
input_data['preOOF'][10000]

'2'

In [6]:
feature.weather.unique()

array(['晴', '曇', '小雨', '雨', '他', '雪'], dtype=object)

In [7]:
feature.sex.unique()

array(['牝', '牡', 'セ'], dtype=object)

In [8]:
feature.surface.unique()

array(['芝', 'ダ'], dtype=object)

In [9]:
feature.course.unique()

array(['右', '左', '他', '直線', '外'], dtype=object)

In [10]:
feature.placeCode.value_counts()

東京    76834
京都    76620
中山    69764
阪神    69536
新潟    49266
小倉    41229
中京    39719
福島    35060
札幌    20139
函館    19042
Name: placeCode, dtype: int64

In [11]:
feature.placeCode.unique()

array(['福島', '東京', '中山', '中京', '京都', '阪神', '札幌', '新潟', '小倉', '函館'],
      dtype=object)

In [12]:
feature.surfaceChanged.value_counts()

0.0    379868
1.0     65026
Name: surfaceChanged, dtype: int64

In [13]:
feature.preMargin.value_counts()

他      310363
クビ      81625
ハナ      33554
アタマ     19352
Name: preMargin, dtype: int64

In [14]:
feature.femaleOnly.unique()

array([1., 0.])

In [15]:
race_info.surface.unique()

array(['芝右', 'ダ右', 'ダ左', '芝左', '障芝ダート', '障芝', '芝右外', '芝右内2周', '芝左外',
       '芝直線', '障芝外-内', '障芝外', '障芝左', '障芝内-外'], dtype=object)

In [16]:
race_info.surface_state.unique()

array(['芝 : 良', 'ダート : 良', 'ダート : 稍重', '芝 : 稍重', '芝 : 良ダート : 良',
       'ダート : 重', '芝 : 稍重ダート : 重', '芝 : 良ダート : 稍重', '芝 : 良ダート : 重',
       '芝 : 稍重ダート : 稍重', 'ダート : 不良', '芝 : 重', '芝 : 不良', '芝 : 不良ダート : 不良',
       '芝 : 重ダート : 不良', '芝 : 重ダート : 重', '芝 : 稍重ダート : 不良'], dtype=object)

In [ ]:
data = feature

race_result=race_result.drop({'order_of_finish','horse_number','sex','age','jockey_id','odds','remark','trainer_id','owner_id','horse_weight'},axis=1)
data = pd.merge(data, race_result, on=['race_id', 'horse_id'],how='left')

race_info=race_info.drop({'race_name','surface','distance','weather','surface_state','place_detail','race_class'},axis=1)
data=pd.merge(data, race_info , on='race_id' , how='left')

data=data.drop({'avgsr4','avgWin4','disavesr','dsl','surfaceScore','jAvgWin4'},axis=1)

In [ ]:
data.head(1)

In [ ]:
def pre_pre2_RunStyle(race_id, horse_id):
    eachHorse=data[data['horse_id']==horse_id].sort_values('date')
    eachHorse=eachHorse.reset_index()
    n = eachHorse[eachHorse['race_id']==race_id].index[0]
    if n==0 :
        return np.nan, np.nan
    elif n==1:
        return eachHorse.at[n-1, 'runningStyle'], np.nan
    else:
        return eachHorse.at[n-1, 'runningStyle'], eachHorse.at[n-2, 'runningStyle']

In [ ]:
PreRunStyle(5673,2011105000)

In [ ]:
for n in range(len(data)):
    n_race_id=data['race_id'].loc[n]
    n_horse_id=data['horse_id'].loc[n]
    pre , pre2 = pre_pre2_RunStyle(n_race_id, n_horse_id)
    data.at[n,'preRunningStyle'] = pre
    data.at[n,'pre2RunningStyle'] = pre2
    

In [ ]:
data[['runningStyle','preRunningStyle','pre2RunningStyle']]

In [ ]:
eachHorse=data[data['horse_id']==2011105000].sort_values('date').reset_index()

In [ ]:
eachHorse=eachHorse.reset_index()

In [ ]:
eachHorse


In [ ]:
eachHorse.loc[4,'index'] 

In [ ]:
eachHorse.loc[5,'index']

In [ ]:
eachHorse.loc[5,'order_of_finish']

In [ ]:

if int(eachHorse.loc[5,'order_of_finish'])==2:
    data.at[74553, 'order_was_2'] =data.at[85566,'order_was_2'] + 1
    data.at[74553, 'order_was_3'] =  data.at[85566, 'order_was_3']

In [ ]:
data[74553:74554]

In [ ]:
len(eachHorse)

In [ ]:
eachHorse.loc[0,'index']

In [ ]:
data.horse_id.unique()

In [ ]:
data[data['race_id']==1160][data['horse_id']==2011105000]['order_was_2'] = 1

In [ ]:
data.at[41551,'order_was_2'] = 1

In [ ]:
data['order_was_2']=0
data['order_was_3']=0
for horse_id in data.horse_id.unique():
    eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
    first_race_idx = eachHorse.loc[0,'index']
    for n in range(len(eachHorse)-1):
        this_race_idx=eachHorse.loc[n,'index']
        next_race_idx = eachHorse.loc[n+1,'index']
        if eachHorse.loc[n,'order_of_finish']==2:
            data.at[next_race_idx, 'order_was_2'] =data.at[this_race_idx,'order_was_2'] + 1
            data.at[next_race_idx, 'order_was_3'] =  data.at[this_race_idx, 'order_was_3']
        elif eachHorse.loc[n,'order_of_finish']==3:
            data.at[next_race_idx, 'order_was_2'] =data.at[this_race_idx,'order_was_2']
            data.at[next_race_idx, 'order_was_3'] =  data.at[this_race_idx,'order_was_3'] + 1
        else :
            data.at[next_race_idx, 'order_was_2'] =data.at[this_race_idx,'order_was_2'] 
            data.at[next_race_idx, 'order_was_3'] =data.at[this_race_idx,'order_was_3']
            
    del eachHorse
    gc.collect()
            
        

In [ ]:
data[data['horse_id']==2012100599].sort_values('date')

In [ ]:
order_was_2_3 = data[['race_id','horse_id','order_was_2','order_was_3']]

In [ ]:
order_was_2_3.to_csv('order_was_2_3.csv')

In [ ]:
data.head()

In [ ]:
import gc

In [ ]:
eachHorse[eachHorse['order_of_finish']==1]['order_of_finish'].count()

In [ ]:
def winTime2_3(horse_id, date):
    eachHorse=data[data['horse_id']==horse_id][data['date']<date]
    order_was_2 = eachHorse[eachHorse['order_of_finish']==2]['order_of_finish'].count()
    order_was_3 = eachHorse[eachHorse['order_of_finish']==3]['order_of_finish'].count()
    return order_was_2 ,order_was_3    

In [ ]:
data['order_was_2']=0
data['order_was_3']=0
for horse_id in data.horse_id.unique():
    eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
    for n in range(len(eachHorse)):
        this_date=eachHorse.loc[n,'date']
        this_idx = eachHorse.loc[n,'index']

        data.at[this_idx, 'order_was_2'] ,data.at[this_idx, 
                                                  'order_was_3'] =  winTime2_3(horse_id,this_date)
    del eachHorse
    gc.collect()

In [ ]:
data[data['horse_id']==2012100599].sort_values('distance')[['distance','finishing_time']]

In [ ]:
data[data['horse_id']==2012100599][data['distance']==1400]['finishing_time'].min()

In [ ]:
#距離変化量
for horse_id in data.horse_id.unique():
    eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
    first_race_idx = eachHorse.loc[0,'index']
    data.at[first_race_idx,'changeDist']=0
    for n in range(1, len(eachHorse)):
        before_race_idx = eachHorse.loc[n-1,'index']
        this_race_idx = eachHorse.loc[n,'index']
        data.at[this_race_idx,'changeDist']=data.at[this_race_idx,
                                                    'distance']-data.at[before_race_idx,'distance']
        
    del eachHorse
    gc.collect()

In [ ]:
eachHorse=data[data['horse_id']==2012100599].sort_values('date').reset_index()

In [ ]:
eachHorse.head()

In [ ]:
eachHorse.loc[0,'index']

In [ ]:
data.at[82538,'changeDist']=0

In [ ]:
data[data['horse_id']==2012100599].sort_values('date')[['distance',
                                                            'changeDist']]

In [ ]:
#前走３ハロン順位


In [ ]:
eachRace = data[data['race_id']==4065].sort_values('last_phase',
                                                   ).reset_index()

In [ ]:
#今回の３ハロン順位結果
for race_id in data.race_id.unique():
    eachRace = data[data['race_id']==
                    race_id].sort_values('last_phase').reset_index()
    for n in range(len(eachRace)) :
        idx = eachRace.loc[n,'index']
        data.at[idx, 'last_phase_order'] =  n+1
    del eachRace
    gc.collect()

In [ ]:
#前回の３ハロン順位
for horse_id in data.horse_id.unique():
    eachHorse=data[data['horse_id']==horse_id].sort_values('date').reset_index()
    first_race_idx = eachHorse.loc[0,'index']
    data.at[first_race_idx,'pre_last_phase_order']=0
    for n in range(1, len(eachHorse)):
        before_race_idx = eachHorse.loc[n-1,'index']
        this_race_idx = eachHorse.loc[n,'index']
        data.at[this_race_idx,'pre_last_phase_order']=data.at[before_race_idx,
                                                              'last_phase_order']
    del eachHorse
    gc.collect()

In [ ]:
data[data['horse_id']==2012102296].sort_values('date')

In [ ]:
data['winRunTimes']=data['winRun']*data['enterTimes']
data['not123times'] = data['enterTimes']-data['winRun'
                                              +'Times']-data['order_was_2']-data['order_was_3']

In [ ]:
data[['enterTimes','winRunTimes','order_was_2','order_was_3','not123times']]

In [ ]:
data.isnull().any(axis=0)

In [ ]:
data['dhweight']= data['dhweight'].fillna(data['dhweight'].mean())

In [ ]:
data[data['earning_money'].isnull()]

In [ ]:
data['jwinper']

In [ ]:
data['jwimper'] = data['jwimper'].fillna(0)